# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from citipy import citipy

# Import API key
from api_keys import g_key


ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
weather_df = pd.read_csv('../output_data/weather_data.csv')
weather_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
# Configure gmaps with API key
gmaps.configure(api_key= g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Latitude", "Longitude"]]

# HINT: be sure to handle NaN values
#census_data_complete = census_data_complete.dropna()
#poverty_rate = census_data_complete["Poverty Rate"].astype(float)
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity (%)"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
weather_df = weather_df.dropna()

filter_weather_df = weather_df[(weather_df["Max Temp (F)"] > 70) & 
                        (weather_df["Max Temp (F)"] < 80) &  
                        (weather_df["Wind Speed (mph)"] < 10) &
                        (weather_df["Cloudiness (%)"] == 0)
                       ]
filter_weather_df

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Max Temp (F),Latitude,Longitude,Wind Speed (mph)
78,78,muscat,0,OM,1584036408,46,73.40,23.61,58.59,2.24
99,99,antri,0,IN,1584036411,22,71.78,26.06,78.21,9.82
166,166,dicabisagan,0,PH,1584036125,83,75.56,17.08,122.42,8.63
230,230,dawei,0,MM,1584036428,73,78.62,14.08,98.20,4.88
358,358,yulara,0,AU,1584036349,25,75.20,-25.24,130.99,5.82
419,419,matara,0,LK,1584036451,81,75.56,5.95,80.54,7.96
459,459,kununurra,0,AU,1584036456,88,75.20,-15.77,128.73,1.97
467,467,mount isa,0,AU,1584036457,88,71.60,-20.73,139.50,8.19
477,477,betioky,0,MG,1584036458,50,78.80,-23.72,44.38,3.80
491,491,aloleng,0,PH,1584036460,72,73.22,16.13,119.78,3.09


In [26]:
filter_weather_df['Hotel'] = ""
filter_weather_df['Hotel lat'] = ""
filter_weather_df['Hotel lng'] = ""

filter_weather_df.head()

C:\Users\lujan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lujan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\lujan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Max Temp (F),Latitude,Longitude,Wind Speed (mph),Hotel,Hotel lat,Hotel lng
78,78,muscat,0,OM,1584036408,46,73.40,23.61,58.59,2.24,,,
99,99,antri,0,IN,1584036411,22,71.78,26.06,78.21,9.82,,,
166,166,dicabisagan,0,PH,1584036125,83,75.56,17.08,122.42,8.63,,,
230,230,dawei,0,MM,1584036428,73,78.62,14.08,98.20,4.88,,,
358,358,yulara,0,AU,1584036349,25,75.20,-25.24,130.99,5.82,,,


In [27]:
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in filter_weather_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        filter_weather_df.loc[index, "Hotel"] = name_address["results"][0]["name"]
        filter_weather_df.loc[index, "Hotel lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        filter_weather_df.loc[index, "Hotel lng"] = name_address["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
filter_weather_df.head()

Missing field/result... skipping.


,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Max Temp (F),Latitude,Longitude,Wind Speed (mph),Hotel,Hotel lat,Hotel lng
78,78,muscat,0,OM,1584036408,46,73.40,23.61,58.59,2.24,Kempinski Hotel Muscat Oman,23.6274,58.272
99,99,antri,0,IN,1584036411,22,71.78,26.06,78.21,9.82,"Taj Usha Kiran Palace, Gwalior",26.2033,78.1673
166,166,dicabisagan,0,PH,1584036125,83,75.56,17.08,122.42,8.63,Villa Hotel and Resort,16.995,122.02
230,230,dawei,0,MM,1584036428,73,78.62,14.08,98.20,4.88,The Mandolis Hotel and Restaurant,14.0974,98.1992
358,358,yulara,0,AU,1584036349,25,75.20,-25.24,130.99,5.82,Outback Pioneer Hotel & Lodge - Ayers Rock Resort,-25.2432,130.99


In [32]:
hotel_df = filter_weather_df[["Hotel", "Hotel lat", "Hotel lng"]]
hotel_df 

,Hotel,Hotel lat,Hotel lng
78,Kempinski Hotel Muscat Oman,23.6274,58.272
99,"Taj Usha Kiran Palace, Gwalior",26.2033,78.1673
166,Villa Hotel and Resort,16.995,122.02
230,The Mandolis Hotel and Restaurant,14.0974,98.1992
358,Outback Pioneer Hotel & Lodge - Ayers Rock Resort,-25.2432,130.99
419,The Doctor's House,5.93662,80.5155
459,Hotel Kununurra,-15.7763,128.738
467,Abacus Motel,-20.6995,139.487
477,,,
491,Birdland Beach Club,16.3867,119.876


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in filter_weather_df.iterrows()]
locations = hotel_df[["Hotel lat", "Hotel lng"]]

In [45]:
# Add marker layer ontop of heat map
# Display Map
locations = filter_weather_df[["Latitude","Longitude"]].astype(float)

figure_layout = {
    'width': '600px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable

markers = gmaps.marker_layer(
    locations, info_box_content= hotel_info)

#markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…